# Numerical Precision

## Imports

In [4]:
%run /home/christopher/.jupyter/config.ipy
import numpy as np

np.set_printoptions(precision=1000)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
def p(x, i):
    if np.bitwise_and(x, np.uint64(2**i)) > 0:
        print(1, end=" ")
    else:
        print(0, end=" ")

def view_bits_int64(x):
    x = x.view(np.uint64)
    for i in range(64):
        p(x, i)
        
def view_bits_float64(x):
    x = x.view(np.uint64)
    print("Sign")
    p(x, 63)
    print("\nExponent")
    for i in range(62, 51, -1):
        p(x, i)

    
    print("\nSignificant 1.")
    for i in range(51, -1, -1):
        p(x, i)    
    print()

In [129]:
view_bits_float64(np.float64(1) + np.finfo(np.float64).eps)
print("Sign = 0 -> positive. Exponent of 1023 - 1023 = 0. Significant of 1.0000...1")
print()
view_bits_float64(np.float64(2))
print("Sign = 0 -> positive. Exponent of 1024 - 1023 = 1. Signficant of 1")

Sign
0 
Exponent
0 1 1 1 1 1 1 1 1 1 1 
Significant 1.
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 
Sign = 0 -> positive. Exponent of 1023 - 1023 = 0. Significant of 1.0000...1

Sign
0 
Exponent
1 0 0 0 0 0 0 0 0 0 0 
Significant 1.
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
Sign = 0 -> positive. Exponent of 1024 - 1023 = 1. Signficant of 1


# Double Precision

Remember that a double precision floating point number is represented like: ![Double precision format](./dp.png).

The significand (the thing that is raised to the power) has an implicit $1$ and the fraction encodes the fractional component,

$$
-1^{\text{sign}} 1.(b_{51}b_{50} ... b_{0})_2 \times 2^{\text{exp}}
$$

In practices, the exponent is represented as an unsigned number and what is used is $exp - 1023$.

This means that the smallest number greater than 1 that can be represented is `1.` followed by `51` zeros and a 1 (and of course an exponent of 0). This is,

$$
1 + \frac{1}{2^{52}} \approx 1 + \frac{1}{4 \times 10^{15}} \approx 1 + \frac{2.5}{10^{16}}
$$

In this case, the exponene
This value is represented by `np.finfo.eps`. We can see that this is roughly right

In [14]:
np.finfo(np.float64).eps

2.220446049250313e-16

What about larger numbers? What is the smallest number larger than say `8` that we can represent (we choose `8` as we know we can represent that exactly with all zeros in the fraction and `3` in the exponent).

We know that this will have the same significand as the smallest number larger than 1, the exponent will just be `3` now. So this is roughly,

$$
1 + \frac{2.5}{10^{16}} * 2^3 = 8 + \frac{8 \times 2.5}{10^{16}}
$$

In [118]:
# Where I found 5 by just slowly increasing until it wasn't exactly 8.
# But anything between 5 and 11 rounds to the same value.
# So it is 8
n_eps = 5
print(np.float64(8) + (n_eps * np.finfo(np.float64).eps))
print(8 + np.float64(2 / 10**15))

8.000000000000002
8.000000000000002


In [134]:
# Therefore the ratio between a+eps / a is constant regardless of a. It is always just 1 + eps
a = 256
print((np.float64(a) + (a * np.finfo(np.float64).eps)) / np.float64(a))
print((np.float64(1) + np.finfo(np.float64).eps) / np.float64(1))

1.0000000000000002
1.0000000000000002


# For what do we have perfect precision

I'm going to use single precision here to keep the numbers somewhat reasonable. This has an **8** bit exponent and **23** bit fraction.

* Between 1 and 2 (exponent of 0) the steps are $2^{-23}$
* Between 2 and 4 (exponent of 1) the steps are $2^{-22}$
* Between $2^n$ and $2^{n+1}$ (exponent of n) the steps are $2^{n - 23}$

In other words, for $n = 23$ the steps are the integers. This is roughly 8 million - 16 million.

In [140]:
print(np.float32(10_000_001))
print(np.float32(10_000_001.6))

10000001.0
10000002.0


In [18]:
print(np.float32(5_000_001))
print(np.float32(5_000_001.3))

5000001.0
5000001.5


The other way to think about it is the eps. This becomes 1 at:

In [21]:
print(1 / np.finfo(np.float32).eps)
print(2**23)

8388608.0
8388608


I still get a weird thrill from doing even simple math and having the computer show me that I did it right...

# Np test

In [12]:
x = np.linspace(0, 1, num=11)
print(x.dtype)
print(x)
widths = np.diff(x)
res = np.finfo(np.float64).resolution
print(widths)
print(widths / widths[0])


float64
[0.                  0.1                 0.2                 0.30000000000000004 0.4                 0.5                 0.6000000000000001  0.7000000000000001  0.8                 0.9                 1.                 ]
[0.1                 0.1                 0.10000000000000003 0.09999999999999998 0.09999999999999998 0.10000000000000009 0.09999999999999998 0.09999999999999998 0.09999999999999998 0.09999999999999998]
[1.                 1.                 1.0000000000000002 0.9999999999999998 0.9999999999999998 1.0000000000000009 0.9999999999999998 0.9999999999999998 0.9999999999999998 0.9999999999999998]


In [16]:
np.isclose(widths / widths[0], 1, rtol=res, atol=0)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True])

In [17]:
np.isclose(np.ones(5), 1, rtol=0, atol=0)

array([ True,  True,  True,  True,  True])